# HttpSensor

În cadrul acestei lecții o să ne creem primul task din cadrul unui DAG folosind un sensor. Acest sensor verifică dacă un anumit site (http) există și este disponibil.

In [ ]:
from airflow import DAG
from datetime import datetime, timedelta


# define the default_args
default_args = {
    "owner": "airflow",
    "retries": 3,
    "retry_delay": timedelta(minutes=5),
    "email_on_failure": False,
    "email": "georgiuandrei05@gmail.com"
}

# Define the DAG
with DAG(
    dag_id = 'test_data_forex_pipeline',
    description='Testing the Data Forex Pipeline from Udemy Course',
    schedule_interval="@daily",
    start_date=datetime(2024, 1, 1),
    catchup=False
    ) as dag:
    pass

Momentan, DAG-ul nostru arată așa. Nu are niciun task în el. Acestui DAG trebuie să îi creem acel task prin care verificăm dacă un API este disponibil. Pentru asta trebuie să importăm acest sensor

In [ ]:
from airflow.providers.http.senors.http import HttpSensor

Acum că ne-am importat acest sensor trebuie să ne creem task-ul. Acest task o să fie creat în acel context de with.

In [ ]:
is_forex_rates_available = HttpSensor()

Sensorul respectiv trebuie să îl atribuim unei variabile. Fiecare task pe care îl creem într-un DAG trebuie să îi atribuim un id (task_id). Acest id trebuie să fie unicat în cadrul DAG-ului din care face parte.

HttpSensor are și alte argumente specifice pe care trebuie să le precizăm. Fiecare sensor și operator au o serie de argumente specifice care trebuie specificate în momentul în care se utilizează aceștia.

In [ ]:
is_forex_rates_available = HttpSensor(
    task_id="is_forex_rates_available"
)

Mai sus avem acel task creat cu sensorul de Http, dar acestuia i-am atribuit doar acel id. Acesta mai are nevoie de anumite argumente pentru a putea funcționa. Pentru a înțelege argumentele, trebuie să înțelegem ce anume se verifică. În cazul de față o să verificăm dacă următorul link este disponibil:

https://gist.github.com/marclamberti/f45f872dea4dfd3eaa015a4a1af4b39b

Link-ul de mai sus reprezintă un link de github în care se găsesc mai multe fișiere json. Acest link o să fie împărțit în partea de host și endpoint

host = https://gist.github.com/

endpoint = marclamberti/f45f872dea4dfd3eaa015a4a1af4b39b

Pentru partea de host, o să ne creem o conexiune specificăm în Airflow. Aceasta se poate crea din UI (o să învățăm cum să facem asta). endpoint-ul trebuie specificat ca și argument la sensor. Partea de host se specifică la sensor ca și acea conexiune care trebuie creată.

În continuare o să ne creem această conexiune în Airflow. Pentru a face asta, din UI, de la secțiunea de "Admin" selectăm "Connections" (pentru a naviga la UI-ul de Airflow trebuie să navigăm la localhost:8080)

<img src="../../ss/airflow-section-03/section-03-ss-01.png">

O să se deschidă o nouă pagină în care se găsesc toate conexiunile din Airflow care există la momentul de față. O să ne creem o nouă conexiune prin apăsarea butonului de +. După ce apăsăm acest buton se va deschide o nouă pagină care o să arate așa:

<img src="../../ss/airflow-section-03/section-03-ss-02.png">

În cadrul acestei pagini trebuie să specificăm următoarele lucruri:

- Conn Id = numele conexiunii. Avem nevoie de această denumire în momentul în care finalizăm task-ul cu sensorul respectiv

- Conn Type = selectăm tipul de conexiune către HTTP

- Host = Aici trebuie să introducem host-ul site-ului HTTP (care este specificat mai sus)

După ce realizăm această conexiune, putem să ne finalizăm task-ul respectiv. Task-ului mai trebuie să îi oferim următoarele argumente:

- http_conn_id = numele conexiunii create

- endpoint = endpoint-ul pe care dorim să îl verificăm

- poke_interval (opțional) = la câte secunde să verifice dacă API-ul este disponibil

- timeout (opțional) = specificăm după câte secunde să se returneze un TimeoutError în cazul în care verificare durează prea mult

In [ ]:
is_forex_rates_available = HttpSensor(
        task_id="is_forex_rates_available",
        http_conn_id="forex_api",
        endpoint="marclamberti/f45f872dea4dfd3eaa015a4a1af4b39b",
        response_check=lambda response: "rates" in response.text,
        poke_interval=5,
        timeout=20
    )

De notat faptul că argumentele poke_interval și timeout sunt dipsonibile pentru orice sensor.

In [ ]:
from airflow import DAG
from airflow.providers.http.sensors.http import HttpSensor
from datetime import datetime, timedelta


# define the default_args
default_args = {
    "owner": "airflow",
    "retries": 3,
    "retry_delay": timedelta(minutes=5),
    "email_on_failure": False,
    "email": "georgiuandrei05@gmail.com"
}

# Define the DAG
with DAG(
    dag_id = 'test_forex_data_pipeline',
    description='Testing the Data Forex Pipeline from Udemy Course',
    schedule_interval="@daily",
    start_date=datetime(2024, 1, 1),
    catchup=False
    ) as dag:
    

    is_forex_rates_available = HttpSensor(
        task_id="is_forex_rates_available",
        http_conn_id="forex_api",
        endpoint="marclamberti/f45f872dea4dfd3eaa015a4a1af4b39b",
        response_check=lambda response: "rates" in response.text,
        poke_interval=5,
        timeout=20
    )

Acum că avem acest task în cadrul DAG-ului putem să îl și testăm. Ca să facem asta putem naviga în UI-ul de Airflow, iar pe pagina de DAGs o să ne apară toate DAG-urle pe care le avem create.

<img src="../../ss/airflow-section-03/section-03-ss-03.png">

În acestă listă se poate observa DAG-ul denumit "test_forex_data_pipeline". Putem observa în drapta lui că avem un buton prin care putem să pornim acest DAG. Acest lucru putem să îl facem (deși nu prea se recomandă) ca să testăm să vedem dacă ne funcționează task-ul respectiv cum ne așteptăm să funcționeze.

<img src="../../ss/airflow-section-03/section-03-ss-04.png">

După ce am pornit acest DAG se poate vedea cum că acesta a rulat cu succes (acel cerc verde pe arată acest lucru, cum că s-a rulat cu succes). Dacă dăm click pe acel DAG putem să vedem mai multe informații desrpe acesta (dar asta o să facem mai încolo când avem mai multe task-uri)